In [1]:
from pybliometrics.scopus import AuthorSearch
from pybliometrics.scopus import AuthorRetrieval
from scholarly import scholarly
from crossref.restful import Works
import pandas as pd
import itertools
import sys
sys.path.append("..")
from rcn_py import orcid
from rcn_py import data_storage
import collections
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download('wordnet')
stop_words = set(stopwords.words("english"))
wnl = WordNetLemmatizer()
ps = PorterStemmer()

import networkx as nx
from pyvis.network import Network

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[

### Scopus

In [2]:
fullname = 'Stefan Verhoeven'
author_last = 'Verhoeven'
author_first = 'Stefan'


In [ ]:
s = AuthorSearch('AUTHLAST('+author_last+') and AUTHFIRST('+author_first+')')
#s = AuthorSearch('Stefan Verhoeven')
author_id = s.authors[0].eid.split('-')[-1]
#s.authors[0]

In [ ]:
au = AuthorRetrieval(author_id)
docs = pd.DataFrame(au.get_documents())

In [ ]:
def get_scopus_data(fullname):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())

In [11]:
search_query_author = scholarly.search_author("Peter Kalverla")
author = scholarly.fill(next(search_query_author))
    
# Get all the publication info of the author
docs = []
for i in range(len(author['publications'])):
    docs.append(author['publications'][i]['bib'])

docs

[{'title': 'The international urban energy balance models comparison project: first results from phase 1',
  'pub_year': '2010',
  'citation': 'Journal of applied meteorology and climatology 49 (6), 1268-1292, 2010'},
 {'title': 'Stable atmospheric boundary layers and diurnal cycles: challenges for weather and climate models',
  'pub_year': '2013',
  'citation': 'Bulletin of the American Meteorological Society 94 (11), 1691-1706, 2013'},
 {'title': 'Single-column model intercomparison for a stably stratified atmospheric boundary layer',
  'pub_year': '2006',
  'citation': 'Boundary-Layer Meteorology 118, 273-303, 2006'},
 {'title': 'Initial results from Phase 2 of the international urban energy balance model comparison',
  'pub_year': '2011',
  'citation': 'International Journal of Climatology 31 (2), 244-272, 2011'},
 {'title': 'Quantifying urban heat island effects and human comfort for cities of variable size and urban morphology in the Netherlands',
  'pub_year': '2011',
  'citatio

In [ ]:
s = AuthorSearch('AUTHLAST(Kalverla) and AUTHFIRST(Peter)')
s

### Google Scholar

In [26]:
search_pub = scholarly.search_pubs('17B. 6 LONG TERM OBSERVATIONS OF LONG WAVE RADIATIVE FLUX DIVERGENCE IN THE STABLE BOUNDARY LAYER OVER LAND')

In [27]:
pubinfo=next(search_pub)
pubinfo

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Long term observations of long wave radiative flux divergence in the stable boundary layer over land',
  'author': ['GJ Steeneveld', 'CDG Zwaaftink'],
  'pub_year': '2008',
  'venue': '… . 18th Symposium on …',
  'abstract': 'During clear and calm nights the atmospheric stability is often too strong to maintain turbulence. Then longwave radiation divergence plays a relatively important role in governing the temperature structure under these conditions in the stable boundary layer. However, in the literature there is no agreement on the sign and the magnitude of near surface longwave radiation divergence under low wind conditions. At the same time we realize that large-scale weather forecast and climate models poorly represent (and mostly'},
 'filled': False,
 'gsrank': 1,
 'pub_url': 'https://ams.confex.com/ams/18BLT/techprogram/paper_139639.htm'

In [ ]:
for doc in docs:
        search_pub = scholarly.search_pubs(doc['title'])
        pub_info = next(search_pub)

In [162]:
search_query_author = scholarly.search_author_id("TOokPzcAAAAJ")

In [163]:
search_query_author

{'container_type': 'Author',
 'filled': ['basics'],
 'scholar_id': 'TOokPzcAAAAJ',
 'source': <AuthorSource.AUTHOR_PROFILE_PAGE: 'AUTHOR_PROFILE_PAGE'>,
 'name': 'Peter Kalverla',
 'url_picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=TOokPzcAAAAJ&citpid=2',
 'affiliation': 'Wageningen University',
 'organization': 8198649032033710874,
 'interests': ['offshore wind energy',
  'atmospheric modelling',
  'meteorology',
  'boundary layer'],
 'email_domain': '@wur.nl',
 'citedby': 218}

In [6]:
search_query_author2 = scholarly.search_author('Peter Kalverla')
scholarly.fill(next(search_query_author2))

{'container_type': 'Author',
 'filled': ['basics',
  'indices',
  'counts',
  'coauthors',
  'publications',
  'public_access'],
 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>,
 'scholar_id': 'TOokPzcAAAAJ',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=TOokPzcAAAAJ',
 'name': 'Peter Kalverla',
 'affiliation': 'Wageningen University',
 'email_domain': '@wur.nl',
 'interests': ['offshore wind energy',
  'atmospheric modelling',
  'meteorology',
  'boundary layer'],
 'citedby': 218,
 'organization': 8198649032033710874,
 'citedby5y': 205,
 'hindex': 8,
 'hindex5y': 7,
 'i10index': 6,
 'i10index5y': 6,
 'cites_per_year': {2015: 4,
  2016: 7,
  2017: 2,
  2018: 16,
  2019: 30,
  2020: 36,
  2021: 48,
  2022: 64,
  2023: 9},
 'coauthors': [],
 'publications': [{'container_type': 'Publication',
   'source': <PublicationSource.AUTHOR_PUBLICATION_ENTRY: 'AUTHOR_PUBLICATION_ENTRY'>,
   'bib': {'title': 'Low-level jets over the North 

In [171]:
def get_scholar_data(fullname, foldername):
    # Search author info by their name 
    # author = scholarly.search_author_id(scholar_id)
    search_query_author = scholarly.search_author(fullname)
    author = scholarly.fill(next(search_query_author))
    
    # Get all the publication info of the author
    docs = []
    for i in range(len(author['publications'])):
        docs.append(author['publications'][i]['bib'])

    title = []
    coauthor_name = []
    abstract = []

    pub_df = pd.DataFrame()
    for doc in docs:
        try:
            search_pub = scholarly.search_pubs(doc['title'])
            pub_info = next(search_pub)
            title.append(pub_info['bib']['title'])
            coauthor_name.append(pub_info['bib']['author'])
            abstract.append(pub_info['bib']['abstract'])
        except:
            continue
    pub_df['title'] = title
    pub_df['authors'] = coauthor_name
    pub_df['abstarct'] = abstract

    pub_df.to_csv(foldername+'/'+fullname+"_scholar.csv")
    return pub_df

    

In [2]:
data_storage.get_scholar_data('Peter Kalverla', "/Users/jennifer/RCSNA/data")

,title,authors,abstarct


### Crossref (ORCID)

In [6]:

orcid_id = orcid.name_to_orcid_id(fullname)
orcid_record = orcid.query_orcid_for_record(orcid_id)
docs = orcid.extract_works_section(orcid_record)

In [7]:
orcid_record

{'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-5821-2060',
  'path': '0000-0002-5821-2060',
  'host': 'orcid.org'},
 'preferences': {'locale': 'en'},
 'history': {'creation-method': 'DIRECT',
  'completion-date': None,
  'submission-date': {'value': 1467114399041},
  'last-modified-date': {'value': 1677393206689},
  'claimed': True,
  'source': None,
  'deactivation-date': None,
  'verified-email': True,
  'verified-primary-email': True},
 'person': {'last-modified-date': {'value': 1648113271690},
  'name': {'created-date': {'value': 1467114399264},
   'last-modified-date': {'value': 1467114399264},
   'given-names': {'value': 'Stefan'},
   'family-name': {'value': 'Verhoeven'},
   'credit-name': None,
   'source': None,
   'visibility': 'public',
   'path': '0000-0002-5821-2060'},
  'other-names': {'last-modified-date': None,
   'other-name': [],
   'path': '/0000-0002-5821-2060/other-names'},
  'biography': None,
  'researcher-urls': {'last-modified-date': None,
   'resear

In [44]:
docs[0]['work-summary'][0]

{'put-code': 70327533,
 'created-date': {'value': 1583795690725},
 'last-modified-date': {'value': 1653871432941},
 'source': {'source-orcid': None,
  'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
   'path': '0000-0001-9884-1913',
   'host': 'orcid.org'},
  'source-name': {'value': 'Crossref'},
  'assertion-origin-orcid': None,
  'assertion-origin-client-id': None,
  'assertion-origin-name': None},
 'title': {'title': {'value': 'Coupling Hydrological models using BMI in eWaterCycle'},
  'subtitle': None,
  'translated-title': None},
 'external-ids': {'external-id': [{'external-id-type': 'doi',
    'external-id-value': '10.5194/egusphere-egu2020-11730',
    'external-id-normalized': {'value': '10.5194/egusphere-egu2020-11730',
     'transient': True},
    'external-id-normalized-error': None,
    'external-id-url': {'value': 'https://doi.org/10.5194/egusphere-egu2020-11730'},
    'external-id-relationship': 'self'}]},
 'url': {'value': 'https://doi.org/10.5

In [8]:
def get_crossref_data_by_orcid(orcid_id):
    # Extract author info using ORCID
    orcid_record = orcid.query_orcid_for_record(orcid_id)
    if orcid_record == False:
        return
    docs = orcid.extract_works_section(orcid_record)

    # Extarct publication info from Crossref
    title_list = []
    doi_list = []
    coauthor_name = []
    coauthor_orcid = []
    abstract = []
    works = Works()
    for doc in docs:
        doi, title = orcid.extract_doi(doc)
        w = works.doi(doi)
        orcid_list, name_list = orcid.get_authors(doi)

        title_list.append(title)
        doi_list.append(doi)
        if w:
            if 'abstract' in w.keys():
                abstract.append(w['abstract'])
            else:
                abstract.append('')
        else:
            abstract.append('')
        coauthor_name.append(name_list)
        coauthor_orcid.append(orcid_list)
        
    pub_df = pd.DataFrame()
    pub_df['title'] = title_list
    pub_df['doi'] = doi_list
    pub_df['authors'] = coauthor_name
    pub_df['abstarct'] = abstract
    pub_df['author_orcid'] = coauthor_orcid

    return pub_df

        

In [9]:
def get_crossref_data_by_name(fullname, folderpath):
    # Extract author info using ORCID
    orcid_id = orcid.name_to_orcid_id(fullname)
    orcid_record = orcid.query_orcid_for_record(orcid_id)
    docs = orcid.extract_works_section(orcid_record)

    # Extarct publication info from Crossref
    title_list = []
    doi_list = []
    coauthor_name = []
    coauthor_orcid = []
    abstract = []
    works = Works()
    for doc in docs:
        doi, title = orcid.extract_doi(doc)
        w = works.doi(doi)
        orcid_list, name_list = orcid.get_authors(doi)

        title_list.append(title)
        doi_list.append(doi)
        if 'abstract' in w.keys():
            abstract.append(w['abstract'])
        else:
            abstract.append('')
        coauthor_name.append(name_list)
        coauthor_orcid.append(orcid_list)
        
    pub_df = pd.DataFrame()
    pub_df['title'] = title_list
    pub_df['doi'] = doi_list
    pub_df['authors'] = coauthor_name
    pub_df['abstarct'] = abstract
    pub_df['author_orcid'] = coauthor_orcid

    pub_df.to_csv(folderpath+'/'+fullname+"_crossref.csv")
    return pub_df

        

In [93]:
df_peter = get_crossref_data_by_name("Peter Kalverla", "/Users/jennifer/RCSNA/data")

In [97]:
df = pd.read_csv("/Users/jennifer/RCSNA/data/Peter Kalverla_crossref.csv")
authors_list = df['author_orcid']

In [10]:
orcid_id = orcid.name_to_orcid_id("Peter Kalverla")

In [11]:
orcid_id

'0000-0002-5025-7862'

In [16]:
orcid_record = orcid.query_orcid_for_record(orcid_id)

In [17]:
def coauthor_data_from_csv(fullname, folderpath):
    orcid_id = orcid.name_to_orcid_id(fullname)
    df = pd.read_csv(folderpath+'/'+fullname+"_crossref.csv")
    authors_list = df['author_orcid']
    frames = []
    frames.append(df)
    for coauthors_id in authors_list:
        coauthors_id = coauthors_id.strip('[')
        coauthors_id = coauthors_id.strip(']')
        coauthors_id = coauthors_id.split(',')
        for id in coauthors_id:
                id = id.strip('\' ')
            # orcid_record = orcid.query_orcid_for_record(id)
            # if orcid_record['person']['addresses']['address']:
            #     country = orcid_record['person']['addresses']['address'][0]['country']['value']
            # else:
            #     country = ''
            # if country == 'NL':
                if id != orcid_id:
                    df2 = data_storage.get_crossref_data_by_orcid(id)
                    
                    if df2 is not None:
                        frames.append(df2)
    new_df = pd.concat(frames)
    new_df2 = new_df.drop_duplicates(subset = ['doi'],keep='first', ignore_index=True)
    new_df2.to_csv(folderpath+'/'+fullname+"_coauthors_allpub.csv")
    return new_df2
   
    

In [3]:
data_storage.coauthor_data_from_csv("Peter Kalverla", "/Users/jennifer/RCSNA/data")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
all_df = pd.read_csv("/Users/jennifer/RCSNA/data/Stefan Verhoeven_crossref.csv")
authors_list = all_df['authors']
res=authors_list[0].strip('[')
res=res.strip(']')
res=res.split(',')
#print(type(res))
res

["'Niels Drost'",
 " 'Rolf Hut'",
 " 'Nick Van De Giesen'",
 " 'Ben van Werkhoven'",
 " 'Jerom P.M. Aerts'",
 " 'Jaro Camphuijsen'",
 " 'Inti Pelupessy'",
 " 'Berend Weel'",
 " 'Stefan Verhoeven'",
 " 'Ronald van Haren'",
 " 'Eric Hutton'",
 " 'Fakhereh Alidoost'",
 " 'Gijs van den Oord'",
 " 'Yifat Dzigan'",
 " 'Bouwe Andela'",
 " 'Peter Kalverla'"]

In [139]:
def get_links_from_csv(fullname, folderpath):
    all_df = pd.read_csv(folderpath+'/'+fullname+"_coauthors_allpub.csv")
    authors_list = all_df['author_orcid']
    link = []
    for coauthors_id in authors_list:
        res = coauthors_id.strip('[')
        res = res.strip(']')
        res = res.split(',')
        
        if len(res) >=2:
            link = link+list(itertools.combinations(res, 2))
    edge_data = pd.DataFrame()
    sources = []
    targets = []
    for l in link:
        sources.append(l[0].strip('\' '))
        targets.append(l[1].strip('\' '))
    edge_data['source'] = sources
    edge_data['target'] = targets
    edge_data.to_csv(folderpath+'/'+fullname+"_coauthors_link.csv")
    return edge_data, link
    

In [147]:
def assign_group_node(fullname, folderpath):
    all_df = pd.read_csv(folderpath+'/'+fullname+"_crossref.csv")
    dois = all_df['doi']
    authors_orcid = all_df['author_orcid']
    authors_name = all_df['authors']

    clusters = orcid.orcid_lda_cluster(dois)
    node_data = pd.DataFrame()
    orcid_id = []
    name = []
    group = []
    for i in range(len(authors_orcid)):
        authors_orcid[i] = authors_orcid[i].strip('[')
        authors_orcid[i] = authors_orcid[i].strip(']')
        authors_orcid[i] = authors_orcid[i].split(',')
        authors_name[i] = authors_name[i].strip('[')
        authors_name[i] = authors_name[i].strip(']')
        authors_name[i] = authors_name[i].split(',')
        
        for j in range(len(authors_orcid[i])):
            orcid_id.append(authors_orcid[i][j].strip('\' '))
            name.append(authors_name[i][j].strip('\' '))
            group.append(clusters[dois[i]])

    node_data['orcid'] = orcid_id
    node_data['name'] = name
    node_data['group'] = group
    
    new_node_data = node_data.drop_duplicates(subset = ['orcid'],keep='last', ignore_index=True)

    return new_node_data
    

In [148]:
node_data = assign_group_node("Peter Kalverla", "/Users/jennifer/RCSNA/data")

/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/1420923647.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/1420923647.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/1420923647.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [79]:
collections.Counter(node_data.group)

Counter({0: 61, 4: 6, 8: 9, 1: 11, 5: 8})

In [81]:
len(node_data['orcid'])

95

In [48]:
links = get_links_from_csv("Stefan Verhoeven", "/Users/jennifer/RCSNA/data")

In [84]:
groups = node_data.groupby("group")["orcid"].apply(list).reset_index()
groups["group"] = groups.index
nodes = node_data.merge(groups, how="left", on=["group"])
nodes

,orcid_x,name,group,orcid_y
0,'0000-0001-9795-7981','Niels Drost',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
1,'0000-0002-5697-5697','Rolf Hut',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
2,'0000-0002-7200-3353','Nick Van De Giesen',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
3,'0000-0002-7508-3272','Ben van Werkhoven',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
4,'0000-0003-0157-4818','Jerom P.M. Aerts',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
...,...,...,...,...
90,'0000-0001-6859-4421','Blaise TF Alako',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
91,'0000-0001-6041-844X','Antoine Veldhoven',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
92,'0000-0001-5351-4361','Sjozef van Baal',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."
93,'0000-0002-6395-1407','Rob Jelier',0,"['0000-0001-9795-7981', '0000-0002-5697-5697'..."


In [154]:
def build_network_by_datafile(fullname, folderpath, outputpath):
    node_data = assign_group_node(fullname, folderpath)
    edge_data, link = get_links_from_csv(fullname, folderpath)
    weights = []
    for i in link:
        weights.append(link.count(i))
    edge_data['weight'] = weights
    
    # getting a group id
    groups = node_data.groupby("group")["orcid"].apply(list).reset_index()
    groups["group"] = groups.index
    
    # finding group id for each node from groups dataframe
    nodes = node_data.merge(groups, how="left", on=["group"])
    nodes["title"] = node_data[["name"]]
    nodes["label"] = node_data[["name"]]
    nodes = nodes.drop("orcid_y", axis=1).set_index("orcid_x")

    # Pyvis network
    N = Network(height=800, width="100%", bgcolor="#222222", font_color="white")
    N.toggle_hide_edges_on_drag(False)
    N.barnes_hut()

    # collecting node attributes for network x
    node_attrs = nodes.to_dict("index")

    # creating a network x graph from dataframes
    graph = nx.from_pandas_edgelist(edge_data, edge_attr=True)
    nx.set_node_attributes(graph, node_attrs)

    lastname = fullname.split(' ')[-1]
    N.from_nx(graph)
    N.show(lastname+'_datafile.html')



In [90]:
build_network_by_datafile("Stefan Verhoeven", "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/2502177070.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/2502177070.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/2502177070.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [155]:
build_network_by_datafile("Peter Kalverla", "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/1420923647.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/1420923647.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_1651/1420923647.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

### Store affiliation-country data?
